# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [175]:
# your code goes here
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)
df

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [182]:
filtered = df[(df['Total Claim Amount']>1000)&(df['Response']=='Yes')].reset_index()
columns = ['Customer','Total Claim Amount', 'Response']
filt = filtered[columns]
filt

,Customer,Total Claim Amount,Response
0,OK31456,1358.400000,Yes
1,YJ16163,1358.400000,Yes
2,GW43195,1027.200000,Yes
3,IP94270,1261.319869,Yes
4,FJ28407,1027.000029,Yes
...,...,...,...
62,FN44127,1176.278800,Yes
63,XZ64172,1324.800000,Yes
64,IX60941,1176.278800,Yes
65,QO62792,1008.000000,Yes


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [250]:
df_yes = df[df['Response'] == 'Yes']
avg_claim_by_policy_gender = df_yes.groupby(['Policy', 'Gender'])['Total Claim Amount'].mean().round(2).reset_index()
avg_claim_by_policy_gender.sort_values(by = 'Total Claim Amount', ascending = False)
#This data shows that the special policy is the one that is costing the company more money when it comes to total claim but it must be an expensive policy. There is no bias between men and women, but it seems that women are owed more than men. 

,Policy,Gender,Total Claim Amount
16,Special L3,F,495.57
6,Personal L1,F,485.13
13,Special L1,M,476.39
12,Special L1,F,471.34
15,Special L2,M,466.86
7,Personal L1,M,464.62
0,Corporate L1,F,463.94
9,Personal L2,M,459.43
11,Personal L3,M,452.95
10,Personal L3,F,447.08


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [256]:
customer_count_by_state = df.groupby('State').size().reset_index(name='Customer')
states_with_more_than_500_customers = customer_count_by_state[customer_count_by_state['Customer'] > 500]
states_with_more_than_500_customers


,State,Customer
0,Arizona,1937
1,California,3552
2,Nevada,993
3,Oregon,2909
4,Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [269]:
df3 = df.groupby(['Education', "Gender"])['Customer Lifetime Value'].agg(['max','min','mean']).round(2).sort_values(by = 'Education', ascending=False)
df3
#Females are more apreciated than men because of their higher education level, this is because women have shown that they are better prepared than men. Also a better education does not matter to the company because highschool is the second biggest category with more people. 

max      min     mean
Education            Gender                            
Master               F       51016.07  2417.78  8157.05
                     M       50568.26  2272.31  8168.83
High School or Below F       55277.45  2144.92  8675.22
                     M       83325.38  1940.98  8149.69
Doctor               F       44856.11  2395.57  7328.51
                     M       32677.34  2267.60  7415.33
College              F       61850.19  1898.68  7748.82
                     M       61134.68  1918.12  8052.46
Bachelor             F       73225.96  1904.00  7874.27
                     M       67907.27  1898.01  7703.60